In [1]:
import torch
import numpy as np

In [2]:
A0 = np.random.randn(10,5)
#A0 = (A0.transpose()).dot(A0)
A0 = torch.from_numpy(A0)
A0.requires_grad = True
A = torch.mm(A0,A0.t())
A
#A.requires_grad = True

tensor([[ 4.3621e+00, -1.3835e+00,  2.0876e+00,  2.2452e+00, -5.3978e+00,
         -2.7148e+00,  5.1779e+00, -3.2429e+00,  3.1859e+00,  2.1629e+00],
        [-1.3835e+00,  1.1233e+00, -1.3666e+00, -4.0351e-01,  8.2396e-01,
          7.5615e-01, -9.4841e-01, -3.8923e-01, -6.4075e-01, -5.6573e-01],
        [ 2.0876e+00, -1.3666e+00,  2.8102e+00,  6.6836e-01, -2.1607e+00,
         -3.7723e-01,  2.6955e+00,  5.8294e-03,  2.1232e+00,  8.1947e-01],
        [ 2.2452e+00, -4.0351e-01,  6.6836e-01,  6.7725e+00, -3.1622e+00,
         -1.5331e+00,  4.1728e+00, -1.0258e+00,  2.4556e+00,  1.9869e-01],
        [-5.3978e+00,  8.2396e-01, -2.1607e+00, -3.1622e+00,  8.7091e+00,
          3.0445e+00, -7.1515e+00,  5.6624e+00, -4.7918e+00, -2.2579e+00],
        [-2.7148e+00,  7.5615e-01, -3.7723e-01, -1.5331e+00,  3.0445e+00,
          2.3264e+00, -2.7092e+00,  2.3217e+00, -1.3182e+00, -1.4961e+00],
        [ 5.1779e+00, -9.4841e-01,  2.6955e+00,  4.1728e+00, -7.1515e+00,
         -2.7092e+00,  8.4633e+0

In [3]:
#from gpytorch

def psd_safe_cholesky(A, upper=False, out=None, jitter=None):
    """Compute the Cholesky decomposition of A. If A is only p.s.d, add a small jitter to the diagonal.
    Args:
        :attr:`A` (Tensor):
            The tensor to compute the Cholesky decomposition of
        :attr:`upper` (bool, optional):
            See torch.cholesky
        :attr:`out` (Tensor, optional):
            See torch.cholesky
        :attr:`jitter` (float, optional):
            The jitter to add to the diagonal of A in case A is only p.s.d. If omitted, chosen
            as 1e-6 (float) or 1e-8 (double)
    """
    try:
        L = torch.cholesky(A, upper=upper, out=out)
        return L
    except RuntimeError as e:
        isnan = torch.isnan(A)
        if isnan.any():
            raise NanError(
                f"cholesky_cpu: {isnan.sum().item()} of {A.numel()} elements of the {A.shape} tensor are NaN."
            )

        if jitter is None:
            jitter = 1e-6 if A.dtype == torch.float32 else 1e-8
        Aprime = A.clone()
        jitter_prev = 0
        for i in range(3):
            jitter_new = jitter * (10 ** i)
            Aprime.diagonal(dim1=-2, dim2=-1).add_(jitter_new - jitter_prev)
            jitter_prev = jitter_new
            try:
                L = torch.cholesky(Aprime, upper=upper, out=out)
                #warnings.warn(f"A not p.d., added jitter of {jitter_new} to the diagonal", NumericalWarning)
                return L
            except RuntimeError:
                continue
        raise e

In [4]:
# trace_A = ((torch.cholesky(A)).diag().log()).sum()
# trace_A.backward()

In [5]:
def log_det(L):
    return (L.diag().log()).sum()
    

In [6]:
L = psd_safe_cholesky(A)
y = log_det(L)
y.backward()
#A0.grad

In [8]:
A

tensor([[ 4.3621e+00, -1.3835e+00,  2.0876e+00,  2.2452e+00, -5.3978e+00,
         -2.7148e+00,  5.1779e+00, -3.2429e+00,  3.1859e+00,  2.1629e+00],
        [-1.3835e+00,  1.1233e+00, -1.3666e+00, -4.0351e-01,  8.2396e-01,
          7.5615e-01, -9.4841e-01, -3.8923e-01, -6.4075e-01, -5.6573e-01],
        [ 2.0876e+00, -1.3666e+00,  2.8102e+00,  6.6836e-01, -2.1607e+00,
         -3.7723e-01,  2.6955e+00,  5.8294e-03,  2.1232e+00,  8.1947e-01],
        [ 2.2452e+00, -4.0351e-01,  6.6836e-01,  6.7725e+00, -3.1622e+00,
         -1.5331e+00,  4.1728e+00, -1.0258e+00,  2.4556e+00,  1.9869e-01],
        [-5.3978e+00,  8.2396e-01, -2.1607e+00, -3.1622e+00,  8.7091e+00,
          3.0445e+00, -7.1515e+00,  5.6624e+00, -4.7918e+00, -2.2579e+00],
        [-2.7148e+00,  7.5615e-01, -3.7723e-01, -1.5331e+00,  3.0445e+00,
          2.3264e+00, -2.7092e+00,  2.3217e+00, -1.3182e+00, -1.4961e+00],
        [ 5.1779e+00, -9.4841e-01,  2.6955e+00,  4.1728e+00, -7.1515e+00,
         -2.7092e+00,  8.4633e+0

In [11]:
w,v = torch.eig(A,eigenvectors=True)


# L = torch.mm(P_cuda,P_cuda.t())
# w,v = torch.eig(L,eigenvectors=True),
# v.mul_(torch.sqrt(torch.sqrt(1./w[:,0])))
# U = torch.mm(v,v.t())
# P_cuda = torch.mm(U,P_cuda)
# print(w)